In [1]:
// Data Types and Functions
using System.Text.RegularExpressions;

#nullable enable
public record struct Operation {
    public string Left;
    public string Right;
    public string Operator;
}

public class Monkey {
    public string Name { get; set; } = null!;
    public double? Value { get; set; }
    public bool? IsIndependent { get; set; }
    public Operation? Operation { get; set; }
}

public static readonly var ValueMonkeyRegex = new Regex(@"(\w+): (\d+)");
public static readonly var OperationMonkeyRegex = new Regex(@"(\w+): (\w+) ([\+\-\/\*]) (\w+)");
public static Monkey ParseMonkey(string line) {
    if (ValueMonkeyRegex.IsMatch(line)) {
        var matchGroups = ValueMonkeyRegex.Match(line).Groups;
        return new Monkey {
            Name = matchGroups[1].Value,
            Value = double.Parse(matchGroups[2].Value),
            IsIndependent = true,
        };
    }
    if (OperationMonkeyRegex.IsMatch(line)) {
        var matchGroups = OperationMonkeyRegex.Match(line).Groups;
        return new Monkey {
            Name = matchGroups[1].Value,
            Operation = new Operation {
                Left = matchGroups[2].Value,
                Operator = matchGroups[3].Value,
                Right = matchGroups[4].Value,
            },
            IsIndependent = null,
        };
    }
    throw new InvalidOperationException($"Could not parse monkey from line {line}");
}

public static Dictionary<string, Monkey> ParseMonkeys(IEnumerable<string> lines) {
    return (
        lines.Select(ParseMonkey)
            .ToDictionary(monkey => monkey.Name)
    );
}

public static double CalculateOperation(double left, string @operator, double right) {
    return @operator switch {
        "+" => left + right,
        "-" => left - right,
        "*" => left * right,
        "/" => left / right,
        _ => throw new InvalidOperationException($"Unknown operator {@operator}"),
    };
}
public static double CalculateValueForMonkey(Dictionary<string, Monkey> monkeys, string monkeyName) {
    var monkey = monkeys[monkeyName];
    if (monkey.Value is null) {
        var leftValue = CalculateValueForMonkey(monkeys, monkey.Operation.Value.Left);
        var rightValue = CalculateValueForMonkey(monkeys, monkey.Operation.Value.Right);
        monkey.Value = CalculateOperation(leftValue, monkey.Operation.Value.Operator, rightValue);
    }
    return monkey.Value.Value;
}

public static bool MonkeyIsIndependentOfHuman(Dictionary<string, Monkey> monkeys, string monkeyName) {
    if (monkeyName == "humn") {
        return false;
    }
    var monkey = monkeys[monkeyName];
    if (monkey.IsIndependent == null) {
        var leftIsIndependent = MonkeyIsIndependentOfHuman(monkeys, monkey.Operation.Value.Left);
        var rightIsIndependent = MonkeyIsIndependentOfHuman(monkeys, monkey.Operation.Value.Right);
        monkey.IsIndependent = leftIsIndependent && rightIsIndependent;
    }
    return monkey.IsIndependent.Value;
}

public static double CalculateValueForHuman(Dictionary<string, Monkey> monkeys, Monkey dependentMonkey, double independentValue) {
    //Console.WriteLine($"{dependentMonkey.Name} {independentValue}");
    if (dependentMonkey.Name == "humn") {
        return independentValue;
    }
    var leftMonkey = monkeys[dependentMonkey.Operation.Value.Left];
    var rightMonkey = monkeys[dependentMonkey.Operation.Value.Right];

    var leftIsIndependent = MonkeyIsIndependentOfHuman(monkeys, leftMonkey.Name);
    var rightIsIndependent = MonkeyIsIndependentOfHuman(monkeys, rightMonkey.Name);
    if (leftIsIndependent && rightIsIndependent) {
        throw new InvalidOperationException($"No human in equation for monkey {dependentMonkey.Name}");
    }
    if (!leftIsIndependent && !rightIsIndependent) {
        throw new InvalidOperationException($"Human on both sides of equation for monkey {dependentMonkey.Name}");
    }
    if (!leftIsIndependent && rightIsIndependent) {
        var rightValue = CalculateValueForMonkey(monkeys, rightMonkey.Name);
        var operation = dependentMonkey.Operation.Value;
        return operation.Operator switch {
            "+" => CalculateValueForHuman(monkeys, leftMonkey, independentValue - rightValue),
            "-" => CalculateValueForHuman(monkeys, leftMonkey, independentValue + rightValue),
            "*" => CalculateValueForHuman(monkeys, leftMonkey, independentValue / rightValue),
            "/" => CalculateValueForHuman(monkeys, leftMonkey, independentValue * rightValue),
            _ => throw new InvalidOperationException($"Unknown operator {@operation.Operator}"),
        };
    }
    if (leftIsIndependent && !rightIsIndependent) {
        var leftValue = CalculateValueForMonkey(monkeys, leftMonkey.Name);
        var operation = dependentMonkey.Operation.Value;
        return operation.Operator switch {
            "+" => CalculateValueForHuman(monkeys, rightMonkey, independentValue - leftValue),
            "-" => CalculateValueForHuman(monkeys, rightMonkey, leftValue - independentValue),
            "*" => CalculateValueForHuman(monkeys, rightMonkey, independentValue / leftValue),
            "/" => CalculateValueForHuman(monkeys, rightMonkey, leftValue / independentValue),
            _ => throw new InvalidOperationException($"Unknown operator {@operation.Operator}"),
        };
    }
    throw new InvalidOperationException();
}

The below script needs to be able to find the current output cell; this is an easy method to get it.


(63,58): warning CS8629: Nullable value type may be null.

(76,69): warning CS8629: Nullable value type may be null.

(88,30): warning CS8629: Nullable value type may be null.



In [2]:
// Read input
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,blwt: dlgr + hrmn
1,qjpw: fncn + pmpf
2,qsph: lfjt * lfnj
3,wdlv: gtlm + spqs
4,lfjg: nrps * bvrb
5,hcvr: jvhm * sflb
6,vclr: 8
7,sspl: lfvv + qntd
8,wlmv: 3
9,gjfq: 3


In [3]:
// Part 1
var monkeys = ParseMonkeys(input);
(long) CalculateValueForMonkey(monkeys, "root")

43699799094202

In [4]:
// Part 2
var monkeys = ParseMonkeys(input);
var rootOperation = monkeys["root"].Operation.Value;
var leftIsIndependent = MonkeyIsIndependentOfHuman(monkeys, rootOperation.Left);
var rightIsIndependent = MonkeyIsIndependentOfHuman(monkeys, rootOperation.Right);
if (leftIsIndependent && rightIsIndependent) {
    throw new InvalidOperationException("No human in equation for monkey root");
}
if (!leftIsIndependent && !rightIsIndependent) {
    throw new InvalidOperationException("Human on both sides of equation for monkey root");
}
if (!leftIsIndependent && rightIsIndependent) {
    var independentValue = CalculateValueForMonkey(monkeys, rootOperation.Right);
    Console.WriteLine($"{(long) CalculateValueForHuman(monkeys, monkeys[rootOperation.Left], independentValue)}");
}
if (leftIsIndependent && !rightIsIndependent) {
    var independentValue = CalculateValueForMonkey(monkeys, rootOperation.Left);
    Console.WriteLine($"{(long) CalculateValueForHuman(monkeys, monkeys[rootOperation.Right], independentValue)}");
}



3375719472770
